In [1]:
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
# for dealing with raster data
import rasterio as rs
from rasterio.plot import show
from rasterio.features import rasterize
from rasterio.mask import mask
# Operating system tasks (e.g. checking if files exists)
import os
# for http requests
import requests
# allows to read data directly from the http request
from io import BytesIO
# for the shape file stuff (and sooo many other things)
import geopandas as gpd
from shapely.geometry import shape
from shapely.ops import unary_union

import imageio
from PIL import Image, ImageDraw, ImageFont

import ee # Earth engine API
import folium

from itertools import chain, combinations

from pprint import pprint

from datetime import datetime, timedelta, UTC

from collections import defaultdict

import tempfile

In [ ]:
def add_ee_layer(self, ee_image_object, vis_params, name, show=True):
    '''
    For viewing the images in an interactive folium map
    '''
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True,
        show=show
    ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer

def get_filter(unique_dict):
    '''
    builds an ee.Filter from the dictionary provided
    '''
    filter_list = [ee.Filter.eq(prop, j) for prop, j in unique_dict.items()]
    return ee.Filter.And(*filter_list)

def unpack_layer_name(nm):
    '''
    builds a dictionary from the layer name provided
    can be passed directly to get_filter()
    '''
    nm_split = nm.split('_')
    return {
        "relativeOrbitNumber_stop": int(nm_split[0]),
        "sliceNumber": int(nm_split[1]),
        "platform_number": nm_split[2],
        "orbitProperties_pass": nm_split[3],
    }

In [ ]:
# Authenticate by logging into your google acct
#ee.Authenticate()
# initialize the API
ee.Initialize()

In [ ]:
'''
Steps:

1. General inquiry in area of interest (AOI)
2. Plot each unique scene to test out different mosaic configurations
3. Choose the mosaic configurations


to cover the region of interest, need to pin down all of the:
relativeOrbitNumber_stop, sliceNumber, platform_number, orbitProperties_pass

'''
gdf = gpd.read_file('shp/saglek_wide.shp', engine='fiona')
#gdf = gpd.read_file('shp/nunatsiavut_coastline.shp', engine='fiona')
gdf = gpd.read_file('shp/scott.shp', engine='fiona')
#gdf = gpd.read_file('file:///Users/ccroberts/Applications/altimetryFit/shapes/cook_wide.shp', engine='fiona')
aoi = ee.Geometry(gdf.geometry[0].__geo_interface__) # a large test region

startDate, endDate = '2015-09-01', '2025-01-01' # short test window

s1_ini = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(aoi)
    .filterDate(startDate, endDate)
    .filter(ee.Filter.eq('instrumentMode', 'IW'))   # Interferometric Wide (IW, 10 m) swath or Extra Wide (EW, 40 m) swath
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH')).select('HH') # choose one: VV or HH
    #.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).select('VV')
).sort('system:time_start')

print(f'{s1_ini.size().getInfo()} images found')
print(f'Available Polarizations: {set(chain.from_iterable(s1_ini.aggregate_array('transmitterReceiverPolarisation').getInfo()))}')
print(f' (note: ignore cross-polarization (HV, VH))')

In [ ]:
# Step 1: define your metadata keys
props = ['relativeOrbitNumber_stop', 'sliceNumber', 'platform_number', 'orbitProperties_pass']
meta = props + ['system:time_start']

# Step 2: get metadata from Earth Engine
prop_values = {
    key: s1_ini.aggregate_array(key).getInfo()
    for key in meta
}

# Step 3: build image-wise metadata dictionaries
s1_meta = [
    {key: prop_values[key][i] for key in meta}
    for i in range(len(prop_values[props[0]]))
]

# Step 4: group by the unique (orbit, slice, platform, pass)
groups = defaultdict(list)
for entry in s1_meta:
    group_key = tuple(entry[key] for key in props)
    groups[group_key].append(entry['system:time_start'])

# Step 5: construct `s1_unique` with first and last timestamps
props_unique = []
for key, times in groups.items():
    s1_entry = {prop: key[i] for i, prop in enumerate(props)}
    times = sorted(times)
    s1_entry['first_date'] = datetime.fromtimestamp(times[0]/1000, tz=UTC)
    s1_entry['last_date'] = datetime.fromtimestamp(times[-1]/1000, tz=UTC)
    s1_entry['count'] = len(times)
    props_unique.append(s1_entry)

# Optional: sort by time range (large to small)
props_unique = sorted(
    props_unique,
    key=lambda d: d['last_date'] - d['first_date']
    )[::-1]

unique_filters = [get_filter({k: filt_dict[k] for k in props}) for filt_dict in props_unique]
unique_imgs = [s1_ini.filter(filt).first() for filt in unique_filters]

print(f'{len(unique_imgs)} unique images found')

In [ ]:
center = aoi.centroid().getInfo()['coordinates'][::-1]
f = folium.Figure(width=600, height=400)
m = folium.Map(location=center, zoom_start=7).add_to(f)

img = unique_imgs[0]
available_bands = img.bandNames().getInfo()
if 'VV' in available_bands and 'HH' in available_bands:
    vizParams = {'min': -25, 'max': 0, 'bands': ['VV', 'HH']}
elif 'VV' in available_bands:
    vizParams = {'min': -25, 'max': 0, 'bands': ['VV']}
elif 'HH' in available_bands:
    vizParams = {'min': -25, 'max': 0, 'bands': ['HH']}
else:
    raise ValueError("No expected bands ('VV' or 'HH') found in image.")

for i, img in enumerate(unique_imgs):
    nm = '_'.join(str(v) for v in [props_unique[i][prop] for prop in props])
    count = props_unique[i]['count']
    firstDate = props_unique[i]['first_date'].date()
    lastDate = props_unique[i]['last_date'].date()
    print(f'{nm}: {count} collected from {firstDate} to {lastDate}')
    m.add_ee_layer(img, vizParams, name=nm, show=False)

# Add the AOI polygon as a GeoJson layer
folium.GeoJson(
    data=aoi.getInfo(),
    name='AOI',
    style_function=lambda x: {
        'color': 'blue',
        'weight': 2,
        'fillOpacity': 0.1
    }
).add_to(m)

m.add_child(folium.LayerControl())

m

In [ ]:
# Pick the layers that will make a nice mosaic (copy the layer names into layer_nms)
# NB: Write the layers in the order you would like them to be place on the map (i.e. reverse order they are appear in layer_nms)
# layer names: orbitNumber_sliceNumber_platform_passDirection (platform means sentinel-1A or sentinel-1B)
# Note: as of 6/17/25, all tracks must be the same (orbitNumber and passDirection must be constant)
layer_nms = ['120_1_B_ASCENDING', '120_2_B_ASCENDING', '149_2_B_ASCENDING', '149_3_B_ASCENDING'] # saglek_wide
layer_nms = ['85_3_A_ASCENDING', '85_2_A_ASCENDING', '85_3_B_ASCENDING', '85_2_B_ASCENDING'] # scott glacier
#layer_nms = ['10_6_A_ASCENDING', '10_5_A_ASCENDING', '10_6_B_ASCENDING', '10_5_B_ASCENDING'] # cook ice shelf

mosaic_size = 2 # Number of images that make up a mosaic
coverage_tol = 0.85 # fraction of image mosaic needs to cover (guesstimate based on map above)

# Optionally, redefine mosaic parameters
# startDate, endDate = '2015-01-01', '2030-01-01' 
# aoi = # optionally redefine the AOI 


In [ ]:
# build gather metadata for each layer_nm

# Example: {'10_5_A_ASCENDING': {'filter': ..., 'first': ..., 'last': ..., 'geometry': ...}}
ic_dict = {nm: s1_ini.filter(get_filter(unpack_layer_name(nm))) for nm in layer_nms}

filter_dict = {
    nm: {
        'filter': get_filter(unpack_layer_name(nm)),
        'times': ic_dict[nm].aggregate_array('system:time_start').getInfo(),
        'geom': ic_dict[nm].geometry()
    }
    for nm in layer_nms
}

In [ ]:
# Group by track first

# Convert to shapely
aoi_shape = shape(aoi.getInfo())
for v in filter_dict.values():
    v['shapely_geom'] = shape(v['geom'].getInfo())

slice_groups = defaultdict(list)
for i, nm in enumerate(layer_nms):
    props = unpack_layer_name(nm)
    key = (props['relativeOrbitNumber_stop'], props['platform_number'], props['orbitProperties_pass'])
    slice_groups[key].append(nm)
    
combo_list = []
# Assemble list of combos to assess
for key in slice_groups.keys():
    count = len(slice_groups[key])
    combo = slice_groups[key]
    if count > mosaic_size: 
        ## code to make new combos by adding how every many more elements necessary
        ## creat combo_list
        print('ERROR: not yet able to handle mosaics containing more than one track')
        print('Select smaller aoi such that single tracks can cover the area')
    elif count <= mosaic_size:
        combo_list.append(tuple(combo))
    else: 
        print('ERROR: unknown')

coverage_sets = []
for combo in combo_list:
    union = unary_union([filter_dict[k]['shapely_geom'] for k in combo])
    coverage = union.intersection(aoi_shape).area / aoi_shape.area
    if coverage > coverage_tol:
        # Intersect time ranges
        print(combo)
        start = max(filter_dict[k]['times'][0] for k in combo)
        end = min(filter_dict[k]['times'][-1] for k in combo)
        if start < end:
            start_dt = datetime.fromtimestamp(start / 1000, tz=UTC).isoformat()
            end_dt = datetime.fromtimestamp(end / 1000, tz=UTC).isoformat()
            coverage_sets.append({'filter_keys': combo, 'start': start_dt, 'end': end_dt, 'coverage': round(float(coverage), 5)})

print(f'Generated {len(coverage_sets)} sets with at least {coverage_tol*100} % coverage')
coverage_sets

In [ ]:
# might stop making a lot of sense from here


# Align slices in each track
tile_sets = []
for s in coverage_sets:
    s1a = {nm: ic_dict[nm] for nm in s['filter_keys']}
    mission_ids = {i: s1a[i].aggregate_array('missionDataTakeID') for i in s1a}
    mission = {i: list(mission_ids[i].getInfo()) for i in s1a}
    common_ids = reduce(lambda a, b: a & b, [set(mission[i]) for i in s1a])
    common_ids_ee = ee.List(list(common_ids))
    s1a = {
        i: s1a[i].filter(ee.Filter.inList('missionDataTakeID', common_ids_ee))
        for i in s1a
    }
    tile_sets.append(s1a)
    
## put together mosaics for perfectly match collections of slices
mosaics = []
for tiles in tile_sets:
    tiles = list(tiles.values())
    img_lists = [col.toList(col.size()) for col in tiles]
    n_imgs = img_lists[0].size().getInfo()
    for i in range(n_imgs):
        imgs = [ee.Image(lst.get(i)) for lst in img_lists]
        mosaic = ee.ImageCollection(imgs).mosaic()
        mosaic_clipped = mosaic.clip(aoi)
        ts = imgs[0].get('system:time_start')  # or grab earliest if you want
        mosaics.append(mosaic_clipped.set('system:time_start', ts))

mosaic_ic = ee.ImageCollection(mosaics).sort('system:time_start')
mosaics_sorted = mosaic_ic.toList(mosaic_ic.size())
mosaics = [ee.Image(mosaics_sorted.get(i)) for i in range(mosaic_ic.size().getInfo())]
dates = [datetime.fromtimestamp(ts / 1000, tz=UTC) for ts in mosaic_ic.aggregate_array('system:time_start').getInfo()]


In [ ]:
# show the first n scenes to make sure it all went went
n = 5

center = aoi.centroid().getInfo()['coordinates'][::-1]
m = folium.Map(location=center, zoom_start=7)


available_bands = img.bandNames().getInfo()
if 'VV' in available_bands and 'HH' in available_bands:
    vizParams = {'min': -25, 'max': 0, 'bands': ['VV', 'HH']}
elif 'VV' in available_bands:
    vizParams = {'min': -25, 'max': 0, 'bands': ['VV']}
elif 'HH' in available_bands:
    vizParams = {'min': -25, 'max': 0, 'bands': ['HH']}
else:
    raise ValueError("No expected bands ('VV' or 'HH') found in image.")

for i, img in enumerate(mosaics[:n]):
    nm = str(dates[i].date())
    m.add_ee_layer(img, vizParams, name=nm)

# Add the AOI polygon as a GeoJson layer
folium.GeoJson(
    data=aoi.getInfo(),
    name='AOI',
    style_function=lambda x: {
        'color': 'blue',
        'weight': 2,
        'fillOpacity': 0.1
    }
).add_to(m)

m.add_child(folium.LayerControl())

m

In [ ]:
region = aoi  # ee.Geometry.Polygon(...)
crs = 'EPSG:3031'

def plot_frame(img):
    
    context_map = Image.open('scott_context.png')
    context_np = np.array(context_map)    

    # Get image date
    date_str = img.date().format('YYYY-MM-dd').getInfo()

    # Get thumbnail in correct CRS
    url = img.getThumbURL({
        'scale': 120, # 10 is native resolution, more like 50-70 may be good
        'format': 'png',
        'crs': crs,
        'min': -25,
        'max': 5,
        'palette': ['black', 'white']  # or omit for grayscale
    })

    response = requests.get(url)
    if response.status_code != 200:
        raise RuntimeError(f"Thumbnail request failed with status {response.status_code}:\n{response.text[:200]}")
    thumb = Image.open(BytesIO(response.content))

    img_np = np.array(thumb)[:, :, 0]

    # Plot
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(img_np, cmap='gray')
    #ax.set_title(date_str, fontsize=14, weight='bold')
    ax.text(
        0.02, 0.04, date_str,  # x, y in axes fraction (0–1)
        transform=ax.transAxes,
        fontsize=10,
        color='black',
        bbox=dict(facecolor='white', edgecolor='black', boxstyle='square,pad=0.3', alpha=0.8)
    )
    ax.text(
        0.98, 0.02, 'Imagery from Sentinel-1',  # x, y in axes fraction (bottom right)
        transform=ax.transAxes,
        fontsize=6,
        color='white',
        ha='right', va='bottom',
    )
    
    # Add context map inset in top-left
    #inset_ax = fig.add_axes([0.14, 0.66, 0.13, 0.13])  # [left, bottom, width, height] # cook
    inset_ax = fig.add_axes([0.77, 0.32, 0.13, 0.13])  # [left, bottom, width, height] # scott
    inset_ax.imshow(context_np)
    inset_ax.axis('off')
    
    ax.axis('off')
    plt.close()
    
    return fig

fig = plot_frame(img)
display(fig)

#fig.savefig('/Users/ccroberts/Desktop/test_frame.png', dpi=500, bbox_inches='tight')

In [ ]:
gif_path = '~/Desktop/scott_movie.gif'


# Temporary directory to store PNGs
with tempfile.TemporaryDirectory() as tmpdir:
    frame_paths = []

    for i, img in enumerate(mosaics):
        print(f"Processing img {i} of {len(mosaics)})", end='\r', flush=True)
        fig = plot_frame(img)
        frame_path = os.path.join(tmpdir, f'frame_{i:03d}.png')
        fig.savefig(frame_path, dpi=500, bbox_inches='tight') # 150 for testing, 750 or 1000 for the final run (will take forever)
        frame_paths.append(frame_path)

    # Write to GIF
    print('\n Writing to gif')
    with imageio.get_writer(gif_path, mode='I', duration=0.75) as writer:
        for frame_path in frame_paths:
            image = imageio.imread(frame_path)
            writer.append_data(image)

print(f"Saved movie to {gif_path}")


In [ ]:
gif_path = '~/Desktop/scott_movie.gif'


# Temporary directory to store PNGs
with tempfile.TemporaryDirectory() as tmpdir:
    frame_paths = []

    for i, img in enumerate(mosaics):
        print(f"Processing img {i} of {len(mosaics)})", end='\r', flush=True)
        fig = None  # predefine to avoid UnboundLocalError
        try:
            fig = plot_frame(img)
        except RuntimeError as e:
            print(f"\nRetrying frame {i} due to error: {e}")
            try:
                fig = plot_frame(img)
            except RuntimeError as e2:
                print(f"Skipping frame {i} after second failure: {e2}")
        if fig is None:
            continue  # skip to next image
        frame_path = os.path.join(tmpdir, f'frame_{i:03d}.png')
        fig.savefig(frame_path, dpi=500, bbox_inches='tight') # 150 for testing, 750 or 1000 for the final run (will take forever)
        frame_paths.append(frame_path)

    # Write to GIF
    print('\n Writing to gif')
    with imageio.get_writer(gif_path, mode='I', duration=0.75) as writer:
        for frame_path in frame_paths:
            image = imageio.imread(frame_path)
            writer.append_data(image)

print(f"Saved movie to {gif_path}")


End of notebook

### scratchwork

In [ ]:
# time sorting mosaicking 

# assemble the image collection for the movie
s1_unique_fin = {i: unpack_layer_name(nm) for i, nm in enumerate(layer_nms)}
unique_filters_fin = [get_filter(s1_unique_fin[i]) for i in s1_unique_fin]

s1_fin = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(aoi)
    .filterDate(startDate, endDate)
    .filter(ee.Filter.eq('instrumentMode', 'IW'))   # Interferometric Wide swath
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .select('VV')  # or 'VH'
).sort('system:time_start')

s1_imgs_all = [s1_fin.filter(filt) for filt in unique_filters_fin]

# group them by time_delta_days, drop scenes that are missing an image
time_window_ms = time_delta_days * 24 * 60 * 60 * 1000  # in milliseconds

# Get all images and their timestamps from each collection
def get_images_and_times(ic):
    imgs = ic.toList(ic.size())
    timestamps = ic.aggregate_array('system:time_start').getInfo()
    return imgs, timestamps

def mosaic_clip(img_list, aoi):
    ic = ee.ImageCollection(img_list)
    mosaic = ic.mosaic()
    mosaic_clipped = mosaic.clip(aoi)
    return mosaic_clipped

img_lists, time_lists = zip(*[get_images_and_times(c) for c in s1_imgs_all])

# Count total images per collection
initial_counts = [len(t) for t in time_lists]

# Loop through reference (col0) and match from others
scenes = []
scene_dates = []
for i0, t0 in enumerate(time_lists[0]):
    center = t0
    scene = [img_lists[0].get(i0)]

    match_found = True
    for j in range(len(layer_nms)):
        diffs = [abs(t - center) for t in time_lists[j]]
        min_diff = min(diffs)
        if min_diff > time_window_ms:
            match_found = False
            break
        min_index = diffs.index(min_diff)
        scene.append(img_lists[j].get(min_index))

    if match_found:
        scenes.append(scene)
        # Convert ms to UTC datetime for display
        scene_dates.append(datetime.utcfromtimestamp(center / 1000).strftime('%Y-%m-%d'))

scenes_clipped = [mosaic_clip(scene, aoi) for scene in scenes]

# Print result
print(f"Matched scenes: {len(scenes)}")
for idx, (date, scene) in enumerate(zip(scene_dates, scenes)):
    print(f"Scene {idx+1}: {date}")

In [ ]:
# show the first n scenes to make sure it all went went
n = 10

center = aoi.centroid().getInfo()['coordinates'][::-1]
m = folium.Map(location=center, zoom_start=7)

# This will give an appropriate color scale for NDVI
vizParams = {'min': -25, 'max': 0, 'bands': ['VV']}

for i, img in enumerate(scenes_clipped[:n]):
    nm = scene_dates[i]
    m.add_ee_layer(img, vizParams, name=nm)

# Add the AOI polygon as a GeoJson layer
folium.GeoJson(
    data=aoi.getInfo(),
    name='AOI',
    style_function=lambda x: {
        'color': 'blue',
        'weight': 2,
        'fillOpacity': 0.1
    }
).add_to(m)

m.add_child(folium.LayerControl())

m